In [1]:
import pandas as pd
import spacy
import glob

# Get statistics from answers

In [2]:
df = pd.read_csv('../data/Dia-Bete/CorpusOficial - Validadas_2021_08_06.csv', sep=',')
# !python -m spacy download pt_core_news_lg
answer_list = df['Resposta para anotação'].values
nlp = spacy.load("pt_core_news_lg")
df.head()

,Questão,Resposta,Resposta para anotação,Resposta oficial (medicina)
0,O jejum intermitente pode ser utilizado tanto ...,Considerando-se que o sobrepeso é um grande fa...,Estar acima do peso pode fazer com que apareça...,Estar acima do peso pode contribuir com o dese...
1,O jejum intermitente pode ser utilizado tanto ...,Considerando-se que o sobrepeso é um grande fa...,Estar acima do peso pode fazer com que apareça...,Estar acima do peso pode contribuir com o dese...
2,Estou fazendo uma dieta low carb e minha glice...,Lembre-se que reduzir muito as calorias não é ...,Lembre que reduzir muito as calorias não é uma...,Lembre que reduzir muito as calorias não é uma...
3,Diabético pode fazer jejum intermitente?,Quem tem diabetes tem que ter uma dieta balanc...,Quem tem diabetes tem que ter uma dieta balanc...,Quem tem diabetes tem que ter uma dieta balanc...
4,"Oi, meu sobrinho tem tipo 1, queria saber mais...",A bomba de infusão de insulina é o melhor trat...,A bomba de infusão de insulina é o melhor trat...,A bomba de infusão de insulina é o melhor trat...


In [3]:
data_map = {}
for answer in answer_list:
    doc = nlp(answer)
#     print(f'Answer: {answer}')
    cur_num_sentences = len(list(doc.sents))
    data_map['num_sentences'] = data_map.get('num_sentences', 0) + cur_num_sentences
#     print(f'Len sentences: {len(list(doc.sents))}')
    for i, sentence in enumerate(doc.sents):
        tokens = [token.text for token in sentence]
        cur_num_tokens = len(tokens)
        data_map['num_tokens'] = data_map.get('num_tokens', 0) + cur_num_tokens
#         print(f'Num tokens: {cur_num_tokens} | Sentence {i}: {sentence.text}')
#         print(f'Tokens: {tokens}')
#     print()
data_map

{'num_sentences': 1403, 'num_tokens': 32090}

# Get statistics from annotations

In [4]:
all_files = glob.glob('../data/Dia-Bete/Dia-Bete_curated_documents_2021-10-15_1746/curation/*/*')
all_files.sort()
df = pd.read_csv('../data/Dia-Bete/all_annotated_dataset_2021_10_15.csv')
total_entities, total_relations = 0, 0
entity_map, relation_map = {}, {}
for cur_file in all_files:
    document_name = cur_file.split('/')[-2]
    file_df = df[df['File'] == document_name]
    entity_set = set([elem for elem in file_df['Entity'] if elem != '_'])
    relation_list = [rel for rel in file_df['Relation'] if rel != '_']
    relation_list = [elem for rel in relation_list for elem in rel.split('|')]
    total_entities += len(entity_set)
    total_relations += len(relation_list)
    # update entity map
    for entity in entity_set:
        upd_entity = entity
        if entity.endswith(']'):
            upd_entity = entity.split('[')[0]
        entity_map[upd_entity] = entity_map.get(upd_entity, 0) + 1
    
    # update relation map
    for relation in relation_list:
        relation_map[relation] = relation_map.get(relation, 0) + 1
        
print(f'Total entities: {total_entities}')
for key, val in entity_map.items():
    print(f'{key:<20} -> {val}')

print(f'\nTotal relations: {total_relations}')
for key, val in relation_map.items():
    print(f'{key:<20} -> {val}')

Total entities: 1645
NonMedicalTreatment  -> 251
DiabetesType         -> 47
Complication         -> 389
GlucoseValue         -> 251
Time                 -> 60
Insulin              -> 25
Duration             -> 18
Set                  -> 22
Test                 -> 67
Symptom              -> 306
Medication           -> 52
Food                 -> 146
Dose                 -> 11

Total relations: 752
causes               -> 75
prevents             -> 47
treats               -> 184
has                  -> 388
diagnoses            -> 52
complicates          -> 3
cases                -> 2
cause                -> 1


In [19]:
main_files = glob.glob('../data/Dia-Bete/Dia-Bete_curated_documents_2021-10-15_1746/curation/*/*')
nutrition_files = glob.glob('../data/Dia-Bete/Dia-Bete_nutrition_curated_documents_2022-01-07_1932/curation/*/*')
main_files.sort()
nutrition_files.sort()
all_files = {'main': main_files, 'nutrition': nutrition_files}
df = pd.read_csv('../data/Dia-Bete/all_annotated_dataset_2022_01_07.csv')
total_entities, total_relations = 0, 0
entity_map, entity_to_token_count, relation_map = {}, {}, {}
for data_name, data_files in all_files.items():
    for cur_file in data_files:
        document_name = cur_file.split('/')[-2]
        file_df = df[df['File'] == f'{data_name}_{document_name}']
        entity_to_token = {}
        for token, entity in zip(file_df['Token'], file_df['Entity']):
            if entity != '_':
                entity_to_token[entity] = entity_to_token.get(entity, '') + ' ' + token 
        entity_set = set([elem for elem in file_df['Entity'] if elem != '_'])
        relation_list = [rel for rel in file_df['Relation'] if rel != '_']
        relation_list = [elem for rel in relation_list for elem in rel.split('|')]
        total_entities += len(entity_set)
        total_relations += len(relation_list)
        # update entity map
        for entity in entity_to_token:
            cur_token = entity_to_token[entity].strip()
            upd_entity = entity
            if entity.endswith(']'):
                upd_entity = entity.split('[')[0]
            entity_map[upd_entity] = entity_map.get(upd_entity, 0) + 1
            if upd_entity not in entity_to_token_count:
                entity_to_token_count[upd_entity] = {cur_token: 1}
            else:
                entity_to_token_count[upd_entity][cur_token] = entity_to_token_count[upd_entity].get(cur_token, 0) + 1

        # update relation map
        for relation in relation_list:
            relation_map[relation] = relation_map.get(relation, 0) + 1
        
print(f'Total entities: {total_entities}')
for key, val in entity_map.items():
    print(f'{key:<20} -> {val}')

print(f'\nTotal relations: {total_relations}')
for key, val in relation_map.items():
    print(f'{key:<20} -> {val}')

Total entities: 2191
Complication         -> 408
DiabetesType         -> 48
NonMedicalTreatment  -> 321
GlucoseValue         -> 300
Time                 -> 71
Insulin              -> 25
Duration             -> 18
Set                  -> 27
Test                 -> 67
Symptom              -> 309
Medication           -> 52
Food                 -> 524
Dose                 -> 21

Total relations: 1084
causes               -> 79
prevents             -> 50
treats               -> 196
has                  -> 700
diagnoses            -> 52
complicates          -> 4
cases                -> 2
cause                -> 1


In [26]:
def print_sorted_labels(labels, length=5):
    for label_tuple in labels[:length]:
        print('Tag: {:<30} | Count: {}'.format(label_tuple[0], label_tuple[1]))

for entity, token_map in entity_to_token_count.items():
    print(f'{entity} - Total: {entity_map[entity]}')
    token_count = list(token_map.items())
    sorted_list = sorted(token_count, key=lambda k: k[1], reverse=True)
    print_sorted_labels(sorted_list, length=10)
    print()

Complication - Total: 408
Tag: diabetes                       | Count: 68
Tag: diabetes diabetes              | Count: 37
Tag: neuropatia diabética           | Count: 33
Tag: polineuropatia sensitiva distal | Count: 27
Tag: diabetes diabetes diabetes     | Count: 16
Tag: perda da sensibilidade         | Count: 13
Tag: dificuldade de sentir o toque  | Count: 13
Tag: neuropatia                     | Count: 8
Tag: cetoacidose diabética          | Count: 7
Tag: disautonomia do sistêma digestório | Count: 7

DiabetesType - Total: 48
Tag: diabetes tipo 2                | Count: 17
Tag: diabetes tipo 1                | Count: 14
Tag: diabetes gestacional           | Count: 7
Tag: tipo 2                         | Count: 6
Tag: tipo 1                         | Count: 4

NonMedicalTreatment - Total: 321
Tag: dieta ideal                    | Count: 46
Tag: dieta balanceada               | Count: 33
Tag: alimentação saudável           | Count: 17
Tag: padrão alimentar saudável      | Count: 16
Tag